## Create database and tables

In [2]:
!conda info --envs

# conda environments:
#
                         /Users/manuel/.julia/conda/3
base                     /Users/manuel/opt/anaconda3
courseragcp              /Users/manuel/opt/anaconda3/envs/courseragcp
iapucp                   /Users/manuel/opt/anaconda3/envs/iapucp
mitxpro                  /Users/manuel/opt/anaconda3/envs/mitxpro
udacity                  /Users/manuel/opt/anaconda3/envs/udacity
udacity-de            *  /Users/manuel/opt/anaconda3/envs/udacity-de



In [26]:
import psycopg2

In [33]:
conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=manuel")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [34]:
cur.execute("DROP DATABASE IF EXISTS sparkifydb")
cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")
cur.execute("GRANT ALL PRIVILEGES ON DATABASE sparkifydb to student;")

# close connection to default database
conn.close()    

In [35]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [36]:
conn.close()

## Test files

In [6]:
import os, glob
import pandas as pd

In [7]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

Song data

In [8]:
song_files = get_files('data/song_data')
len(song_files)

71

In [14]:
df = pd.concat([pd.read_json(file, lines = True) for file in song_files])

In [15]:
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
0,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
0,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


In [16]:
len(df), df.song_id.nunique(), df.artist_id.nunique()

(71, 71, 69)

Song table

In [40]:
song_data = df.get(['song_id', 'title', 'artist_id', 'year', 'duration'])
song_data.head()

,song_id,title,artist_id,year,duration
0,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
0,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
0,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608
0,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
0,SOQHXMF12AB0182363,Young Boy Blues,ARGSJW91187B9B1D6B,0,218.77506


In [41]:
song_data.shape[0], song_data.song_id.nunique()

(71, 71)

[]

In [43]:
try:
    os.mkdir('tables')
except Exception as e:
    print(e)

[Errno 17] File exists: 'tables'


In [48]:
def insert_df_to_table(df, temp_file, connection, table_name):
    """
    insert a dataframe into a postgres table
    df: dataframe with data
    temp_file: local path to checkpoint the dataframe
    connection: connection object to the database
    table_name: table to insert the data
        
    based on implementation from Naysan Saran at
    https://naysan.ca/2020/06/21/pandas-to-postgresql-using-psycopg2-copy_from/
    """
    
    #save values to temp file
    df.to_csv(temp_file, header=False, index = False)
    
    #load temp file
    file = open(temp_file, 'r')
    
    cursor = connection.cursor()
    
    try:
        
        #copy data from file to postgres table
        cursor.copy_from(file, table_name, sep = ',')
    
    except (Exception, psycopg2.DatabaseError) as error:

        #print error if completed
        print("Error: %s" % error)
        
        #remove temp file, rollback and close cursor
        os.remove(temp_file)
        conn.rollback()
        cursor.close()
        
        return
    
    #print message if completed
    print("copy_from_file() done")
    
    #remove temp file and close cursor
    cursor.close()
    os.remove(temp_file)

Connection and insert with copy

In [46]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")

insert_df_to_table(song_data, 'tables/songs.csv', conn, 'songs')

copy_from_file() done


Artist table

In [21]:
artist_data = df.get(['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude'])
artist_data.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
0,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
0,ARXR32B1187FB57099,Gob,,NaN,NaN
0,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
0,ARGSJW91187B9B1D6B,JennyAnyKind,North Carolina,35.21962,-80.01955


In [24]:
artist_data.shape[0], artist_data.drop_duplicates().shape[0], artist_data['artist_id'].nunique()

(71, 69, 69)

In [50]:
artist_data.artist_id.value_counts()

ARNTLGG11E2835DDB9    2
ARD7TVE1187B99BFB1    2
ARPFHN61187FB575F6    1
ARIG6O41187B988BDD    1
ARH4Z031187B9A71F2    1
                     ..
ARNF6401187FB57032    1
ARBEBBY1187B9B43DB    1
AR0IAWL1187B9A96D0    1
AR558FS1187FB45658    1
ARBGXIG122988F409D    1
Name: artist_id, Length: 69, dtype: int64

In [49]:
artist_data.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
0,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
0,ARXR32B1187FB57099,Gob,,NaN,NaN
0,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
0,ARGSJW91187B9B1D6B,JennyAnyKind,North Carolina,35.21962,-80.01955


In [51]:
artist_data.query("artist_id == 'ARNTLGG11E2835DDB9'")

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARNTLGG11E2835DDB9,Clp,,NaN,NaN
0,ARNTLGG11E2835DDB9,Clp,,NaN,NaN


In [52]:
artist_data.query("artist_id == 'ARD7TVE1187B99BFB1'")

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN


## Test reading with pandas

In [53]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
sql = "select * from songs;"

In [55]:
dat = pd.read_sql(sql, conn)

,song_id,title,artist_id,year,duration
